# Test psql

In [1]:
import psycopg2

In [10]:
conn = psycopg2.connect(dbname='sensorhub')

In [11]:
cur = conn.cursor()

In [12]:
cur.execute('SELECT * FROM sensorlog')

In [13]:
cur.fetchall()

[]

In [6]:
cur.execute('DELETE FROM sensorlog')

In [7]:
conn.commit()

In [8]:
cur.close()

In [9]:
conn.close()

# Test sensorhub

In [4]:
import smbus

DEVICE_BUS = 1
DEVICE_ADDR = 0x17

TEMP_REG = 0x01
LIGHT_REG_L = 0x02
LIGHT_REG_H = 0x03
STATUS_REG = 0x04
ON_BOARD_TEMP_REG = 0x05
ON_BOARD_HUMIDITY_REG = 0x06
ON_BOARD_SENSOR_ERROR = 0x07
BMP280_TEMP_REG = 0x08
BMP280_PRESSURE_REG_L = 0x09
BMP280_PRESSURE_REG_M = 0x0A
BMP280_PRESSURE_REG_H = 0x0B
BMP280_STATUS = 0x0C
HUMAN_DETECT = 0x0D

In [5]:
bus = smbus.SMBus(DEVICE_BUS)

In [6]:
bus.read_byte_data(DEVICE_ADDR, TEMP_REG)

21

In [8]:
aReceiveBuf = []

aReceiveBuf.append(0x00) # 占位符

for i in range(TEMP_REG,HUMAN_DETECT + 1):
    aReceiveBuf.append(bus.read_byte_data(DEVICE_ADDR, i))

if aReceiveBuf[STATUS_REG] & 0x01 :
    print("Off-chip temperature sensor overrange!")
elif aReceiveBuf[STATUS_REG] & 0x02 :
    print("No external temperature sensor!")
else :
    print("Current off-chip sensor temperature = %d Celsius" % aReceiveBuf[TEMP_REG])


if aReceiveBuf[STATUS_REG] & 0x04 :
    print("Onboard brightness sensor overrange!")
elif aReceiveBuf[STATUS_REG] & 0x08 :
    print("Onboard brightness sensor failure!")
else :
    print("Current onboard sensor brightness = %d Lux" % (aReceiveBuf[LIGHT_REG_H] << 8 | aReceiveBuf[LIGHT_REG_L]))

print("Current onboard sensor temperature = %d Celsius" % aReceiveBuf[ON_BOARD_TEMP_REG])
print("Current onboard sensor humidity = %d %%" % aReceiveBuf[ON_BOARD_HUMIDITY_REG])

if aReceiveBuf[ON_BOARD_SENSOR_ERROR] != 0 :
    print("Onboard temperature and humidity sensor data may not be up to date!")

if aReceiveBuf[BMP280_STATUS] == 0 :
    print("Current barometer temperature = %d Celsius" % aReceiveBuf[BMP280_TEMP_REG])
    print("Current barometer pressure = %d pascal" % (aReceiveBuf[BMP280_PRESSURE_REG_L] | aReceiveBuf[BMP280_PRESSURE_REG_M] << 8 | aReceiveBuf[BMP280_PRESSURE_REG_H] << 16))
else :
    print("Onboard barometer works abnormally!")

if aReceiveBuf[HUMAN_DETECT] == 1 :
    print("Live body detected within 5 seconds!")
else:
    print("No humans detected!")

Current off-chip sensor temperature = 21 Celsius
Current onboard sensor brightness = 20 Lux
Current onboard sensor temperature = 24 Celsius
Current onboard sensor humidity = 61 %
Current barometer temperature = 24 Celsius
Current barometer pressure = 101517 pascal
No humans detected!


In [12]:
import time

In [9]:
aaReceiveBuf[HUMAN_DETECT]

0

In [13]:
date = time.strftime('%Y-%m-%d %H:%M:%S')
tempOffChip = str(aReceiveBuf[TEMP_REG])
brightness = str((aReceiveBuf[LIGHT_REG_H] << 8 | aReceiveBuf[LIGHT_REG_L]))
tempOnChip = str(aReceiveBuf[ON_BOARD_TEMP_REG])
humidity = str(aReceiveBuf[ON_BOARD_HUMIDITY_REG])
tempBarometer = str(aReceiveBuf[BMP280_TEMP_REG])
pressure = str(aReceiveBuf[BMP280_PRESSURE_REG_L] | aReceiveBuf[BMP280_PRESSURE_REG_M] << 8 | aReceiveBuf[BMP280_PRESSURE_REG_H] << 16)
humanDetected = str(aReceiveBuf[HUMAN_DETECT])

In [3]:
date, tempOffChip, brightness, tempOnChip, humidity, tempBarometer, pressure, humanDetected

NameError: name 'date' is not defined

In [ ]:
(Date timestamp, TempOffchip int, Brightness float8, TempOnChip int, Humidity int, TempBaromter int, Pressure float8, HumanDetected bool)

In [41]:
cur.execute("INSERT INTO sensorlog (Date, TempOffchip, Brightness, TempOnChip, Humidity, TempBaromter, Pressure, HumanDetected) values (%s, %s, %s, %s, %s, %s, %s, %s)",(date, tempOffChip, brightness, tempOnChip, humidity, tempBarometer, pressure, humanDetected))

In [48]:
d = {'bonjour': 1,
       'connard': 2.0}

In [45]:
','.join(d.values())

TypeError: sequence item 0: expected str instance, int found

In [38]:
l = [1,325,6]

In [54]:
f'l={*d.keys(),}'

"l=('bonjour', 'connard')"

# Test timer

In [54]:
import threading

In [57]:
def infinite_loop():
    for i in range (100):
        print('coucou')
        time.sleep(1)

In [63]:
thread = threading.Thread(target=infinite_loop)

In [64]:
thread.start()
time.sleep(0.5)
print('fuck you')

coucou
fuck you
coucou
coucou
coucou
coucou
coucou
coucou
coucou
coucou
coucou


In [1]:
from threading import Timer
import time
import sched

In [2]:
def long_function_timer(i):
    if i<5: 
        t = Timer(10,long_function_timer,[i+1])
        t.start()
    print('starting at :',time.time() )
    time.sleep(2)
    
    print('stopping at :',time.time() )
    
    

In [3]:
long_function_timer(0)

starting at : 1636058333.044665
stopping at : 1636058335.062239
starting at : 1636058343.066123
stopping at : 1636058345.0742104
starting at : 1636058353.080911
stopping at : 1636058355.0911896
starting at : 1636058363.091758
stopping at : 1636058365.0997896
starting at : 1636058373.1100118
stopping at : 1636058375.117895
starting at : 1636058383.125869
stopping at : 1636058385.1322


In [57]:
import sched

In [61]:
def long_function_sched(s, i):
    if i<10: 
        s.enter(10,1,  long_function_sched, argument=(s,i+1,))
    print('starting at :',time.time() )
    time.sleep(2)
    
    print('stopping at :',time.time() )
    
    

In [62]:
s = sched.scheduler(time.time, time.sleep)

In [63]:
long_function_sched(s, 0)

starting at : 1636065298.745115
stopping at : 1636065300.7536552


# Test logger

In [1]:
from loguru import logger

In [4]:
logger.add('test.log', enqueue=True)

2

In [3]:
logger.info('hello world')

2021-10-24 22:27:22.855 | INFO     | __main__:<module>:1 - hello world


# Test sensor connector class

In [20]:
%load_ext autoreload
%autoreload 2
from sensor_connector import SensorConnector

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
sensorConnector = SensorConnector()

In [23]:
sensorConnector.get_all_sensors()

2021-11-04 22:38:26.212 | INFO     | sensor_connector:get_all_sensors:79 - Starting sensors data aquisition
2021-11-04 22:38:26.247 | INFO     | sensor_connector:get_all_sensors:93 - Sensors Data aquisition finished


(21, 21, 24, 63, 24, 1.01582, 1)

In [50]:
starting_time = time.strftime('%d_%m_%H:%M')


In [51]:
f'log/sensor_log_{starting_time}.log'

'log/sensor_log_04_11_23:28.log'

# Test PsqlConnector

In [1]:
%load_ext autoreload
%autoreload 2
from psql_connector import PsqlConnector

In [5]:
psqlConnector = PsqlConnector('sensorhub')

In [8]:
psqlConnector.delete_content('sensorlog')

Are you sure that you want to delete of table "sensorlog" in database "sensorhub" ?  (y, n) [n]: 
Delete aborted.
